In [7]:
import numpy as np
from scipy import linalg as la
from scipy.sparse import dok_matrix

In [52]:
""" Problem 1 """

def get_A(N, filename = '/Users/Deehrlich/Desktop/BootCamp2017/Computation/Wk4_DifIntOpt/matrix.txt'):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    S = dok_matrix((N, N))
    
    for line in lines:
        line = line.strip().split()
        try: 
            i = int(line[0])
            j = int(line[1])
            S[i,j] = 1
        except:
            pass
        
    return S

A = get_A(8)
print(A.toarray())

[[ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  1.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]]


In [123]:
""" Problem 2 """

def get_K(A):
    Am = A.copy()
    rowsum = Am.sum(axis=1)
    Am[np.where(rowsum == 0)[0],:] = 1
    D = np.diagflat(Am.sum(axis=1))
    return (la.inv(D)@Am).T

K = get_K(A)
print(K)

[[ 0.          1.          0.125       0.33333333  0.33333333  0.5         1.
   1.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.33333333  0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.125       0.          0.33333333  0.          0.
   0.        ]
 [ 0.          0.          0.125       0.33333333  0.33333333  0.5         0.
   0.        ]
 [ 1.          0.          0.125       0.          0.          0.          0.
   0.        ]]


In [129]:
""" Problem 3 """

def steadystate(A, N= None, d= 0.85, tol = 1e-5):
    
    K = get_K(A)
    A = A.toarray()
    n, n = A.shape 
    
    if N != None:
        K = K[:N,:N]
    else:
        N = n
    
    pt = np.random.rand(N,1)
    error = 1
    while error > tol:
        pt1 = d*K@pt + (1-d)/N
        error = np.linalg.norm(pt1 - pt)
        pt = pt1
        
    return pt

print(steadystate(A))

[[ 0.43871522]
 [ 0.02171029]
 [ 0.02786154]
 [ 0.02171029]
 [ 0.02171029]
 [ 0.02786154]
 [ 0.04585394]
 [ 0.39461609]]


In [140]:
""" Problem 4 """

def eig_SS(A, N = None, d = 0.85):
    
    K = get_K(A)
    A = A.toarray()
    n, n = A.shape 

    if N != None:
        K = K[:N,:N]
    else:
        N = n
        
    pt = np.random.rand(N,1)
    
    B = d*K + (1-d)/N
    eigs, vecs = la.eig(B)
    ss = vecs[:,0]/np.sum(vecs[:,0])
    return ss

eig_ss = eig_SS(A)
print(eig_ss)

[ 0.43869288  0.02171029  0.02786154  0.02171029  0.02171029  0.02786154
  0.04585394  0.39459924]


In [160]:
""" Problem 5 """

def NCAA(filename = '/Users/Deehrlich/Desktop/BootCamp2017/Computation/Wk4_DifIntOpt/ncaa2013.csv'):

    games = np.empty(shape=[0, 2])
    with open(filename, 'r') as ncaafile:
        ncaafile.readline()
        for line in ncaafile:
            teams = line.strip().split(',')
            games = np.append(games, [[teams[0], teams[1]]], axis=0)
        
    names = np.unique(games)
    index = {k: v for v, k in enumerate(names)}
    A = dok_matrix((len(names), len(names)))
    for i in range(len(games[:,0])):
        A[index[games[i,1]],index[games[i,0]]] = 1
        
    ranks = steadystate(A, d= 0.7)
    sort = np.argsort(-ranks.T)
    return ranks[sort], names[sort]

sortedranks, sortednames = NCAA()
print(sortednames[:,:5])

[['Duke' 'Butler' 'Louisville' 'Illinois' 'Indiana']]
